## Dialogue paraphrase

#### 1. Install the environment by conda, then switch ipython kernel to "sentence_paraphrase"

In [ ]:
# install environment
!conda env create -f environment.yml
# switch the ipython kernel to "sentence_paraphrase"

#### 2. Download the model and data for paraphrase, and the data for training model.
* The datasets we use include DNLI, EMNLP, MSC, peronachat and WOW.

In [ ]:
# download model and training/inference data
!bash script/download.sh

#### 3. Generate the the paraphrase
* We choose test data in EMNLP_dataset for sentence paraphrase, by randomly choose an utterance from one of the speaker as the target for paraphrase.
* If you want to use your own data, please followed the format and save it to a json file:  
`{"prev": "A: ... B: ... A:", "cur": "...", "future": "B: ... A: ..."}, {...}`
* In which "prev" means previous dialogue, "cur" means the utterance for paraphrase, and "future" means future dialogue. Each utterance should have prefix "A: " or "B: " to indicate the speaker. Note that "A: " or "B: " for the utterance for paraphrase should be place at the end of "prev".
* you can simply change the seed for parse_data.py, to generate a different paraphrase file.

In [ ]:
%cd script
# parse the EMNLP data(already done, you can change the seed for different paraphrase file)

#!python parse_data.py --seed 314

# generate paraphrase
!CUDA_VISIBLE_DEVICES=2 python3 generate_paraphrase.py \
    --num_passes 2 \
    --future_gradient_iters 8 \
    --original_gradient_iters 8 \
    --future_lr 6e-1 \
    --original_lr 6e-1 \
    --future_ratio 0.15 \
    --original_ratio 0.6 \
    --top_k 5 \
    --temperature 0.7 \
    --output_file "./script_data/origin.json"

!CUDA_VISIBLE_DEVICES=1 python3 ranking.py \
  --hyps_file "./script_data/origin.json"  \
  --output_file "../result/paraphrase_result.txt" \
  --original_data_file "./script_data/testData_withHint.json"

%cd ..

#### 4. Evaluation
* We employ BLEU, ROUGE-L, and BERTScore metrics to evaluate the dissimilarity between paraphrased sentences and their original counterparts. A lower score indicates a more varied paraphrase.
* We also train a NSP model for measuring the quality between previous dialogue and future dialogue. We use this model to show the quality of paraphrase by nsp score1, which use the combination of the previous dialogue and paraphrase to predict future dialogue; and nsp score2, which use previous dialogue to predict the combination of paraphrase and future dialogue.

In [ ]:
#evaluation
%cd script
!python view_result.py --predicted_file ../result/paraphrase_result.txt
%cd ..